In [14]:
import cv2
import numpy as np
from tqdm import tqdm
from random import shuffle
import os

In [15]:
TRAIN_DIR = "D:\Python tutorial videos\DATA SETS\\train"
TEST_DIR = "D:\Python tutorial videos\DATA SETS\\test1"
IMG_SIZE = 50
LR = 1e-3

MODEL_NAME = "dogs vs cats - {}-{}-model".format(LR, "2CONV-basic")

In [16]:
def label_image(img):
    word_label = img.split(".")[-3]
    if word_label == "cat": return [1,0]
    elif word_label == "dog": return [0,1]
    

In [17]:
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_image
        path = os.path.join(TRAIN_DIR, img)
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (IMG_SIZE, IMG_SIZE))
        training_data.append([np.array(img), np.array(label)])
        
    shuffle(training_data)
    np.save("training_data.npy", training_data)
    return training_data

In [18]:
def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR, img)
        img_num = img.split(".")[0]
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (IMG_SIZE, IMG_SIZE))
        
        testing_data.append([np.array(img), img_num])

    np.save("test_data.npy", testing_data)
    return testing_data

In [19]:
train_data = create_train_data()


100%|███████████████████████████████████████████████████████| 25000/25000 [03:25<00:00, 121.75it/s]


In [9]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
import tflearn.datasets.mnist as mnist


# Building convolutional convnet
convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')
# http://tflearn.org/layers/conv/
# http://tflearn.org/activations/
convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')

AttributeError: module 'tensorflow' has no attribute 'scalar_summary'

In [9]:
if os.path.exists("{}.meta".format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print("MODEL loaded !!!")

In [10]:
train = train_data[:-500]
test = train_data[-500:]


In [13]:
X = np.array([i[0] for i in train]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
Y = [i[1] for i in train]


test_x = np.array([i[0] for i in test]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
test_y = [i[1] for i in test]

In [15]:
model.fit({"input":X}, {"targets":Y}, n_epoch=3, validation_set=({"input":test_x}, {"targets":test_y}),
         snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

---------------------------------
Run id: dogs vs cats - 0.001-2CONV-basic-model
Log directory: log/
Instructions for updating:
Please switch to tf.summary.FileWriter. The interface and behavior is the same; this is just a rename.
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'NoneType' object has no attribute 'name'
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'dict' object has no attribute 'name'
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'NoneType' object has no attribute 'name'
Instructions for updating:
Please switch to tf.summary.merge.
Instructions for updating:
Please switch to tf.summary.merge.
Instructions for updating:
Please switch to tf.summary.merge.
Instructions for updating:
Please switch to tf.summary.merge.
---------------------------------
Training samples: 24500
Validation samples: 500
--


ValueError: setting an array element with a sequence.